<div style="direction:rtl;text-align:center"><img src="https://mohammadkh.ir/github/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Object Detection</div></h1>

In [1]:
import cv2
import numpy as np

# Create Function

#### Load img and Preoroccess

In [2]:
def load_data_and_preprocess(address):
    img = cv2.imread(address)
    h, w = img.shape[:2]
    # blob --> fun perproccess img
    # swapRB -> R <-> B
    preprocessed_image = cv2.dnn.blobFromImage(img, scalefactor= 1/255, size= (416, 416), 
                                               swapRB = True, crop = False)
    return img, preprocessed_image, h, w

#### Load Model & label

In [3]:
def read_model_and_label(label_address, weights_address, config_address):
    # 80 object detect yolo3
    labels = open(label_address).read().strip().split("\n")
    net = cv2.dnn.readNet(weights_address, config_address)
    
    return labels, net

#### Predict Model

In [4]:
def inference(pre_processed_image, net):
    net.setInput(pre_processed_image)
    output_layers = ["yolo_82", "yolo_94", "yolo_106"]
    predictions = net.forward(output_layers)

    return predictions

#### proccess OutPut Model

In [5]:
def post_processing(predictions, w, h):
    # output model 85 array columes --> 4 loc(x_center,y_center,w,h) + P0 (dec obg) + P (80 objects)
    # loc out model % --> %*shpe = loc
    
    classIDs = []
    confidences = []
    boxes = []

    for layer in predictions:                 # all detect object (rows)
        for detected_object in layer:         # 85 col
            scores = detected_object[5:]      # P (80 object)
            classID = np.argmax(scores)       # select P obj max
            confidence = scores[classID]      # read Pmax
            
            # loc box
            if confidence > 0.6:
                box = detected_object[0:4]*np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - width/2)
                y = int(centerY - height/2)

                # add to list parms
                classIDs.append(classID)
                confidences.append(float(confidence))
                boxes.append([x, y, int(width), int(height)])
                
    return classIDs, confidences, boxes

#### Show OutPut

In [6]:
colors = np.random.uniform(0, 255, size= (80, 3))
# print(colors)

def show_results(img, classIDs, confidences, boxes, labels):
    cv2.namedWindow("image",cv2.WINDOW_NORMAL)
    #cv2.resizeWindow('image', 600,600)
    
    #  NMSBoxes --->  select one box for multi box in area ---> get prm box
    idxs = cv2.dnn.NMSBoxes(bboxes=boxes, scores=confidences, score_threshold=0.3, nms_threshold=0.5)

    for i in idxs.flatten():
        (x, y) = boxes[i][0], boxes[i][1]
        (w, h) = boxes[i][2], boxes[i][3]

        # draw rectangle obj
        cv2.rectangle(img, (x, y), (x+w, y+h), colors[i], 10)
        
        # show label and confidence
        ## text write
        text = "{} :  {:.2f}".format(labels[classIDs[i]], confidences[i])
        ##  draw rectangle bg text
        cv2.rectangle(img, (x, y+45), (x+200, y), colors[i], -1)
        ## put text 
        cv2.putText(img, text, (x+10, y+30), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)
        
    cv2.imwrite("pic_yolo_1.jpg", img)
    cv2.imshow("image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# detected

In [8]:
img, pre_img, h, w = load_data_and_preprocess(r"__data/image/dc.jpg") 
# img, pre_img, h, w = load_data_and_preprocess(r"__data/image/family.jpg") 

labels, net = read_model_and_label(r"__data/yolov3/coco.names",
                                   r"__data/yolov3/yolov3.weights",
                                   r"__data/yolov3/yolov3.cfg")

predictions = inference(pre_img, net)
                                   
classIDs, confidences, boxes = post_processing(predictions, w, h)
                                   
show_results(img, classIDs, confidences, boxes, labels)


<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Object Detection</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>